In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import json
import seaborn as sns

In [2]:
test = pd.read_csv('./air_quality/PRSA_Data_Dongsi_20130301-20170228.csv')
test['datetime'] = pd.to_datetime(test[['year', 'month', 'day', 'hour']])
# test.head()
test['datetime'].isna().sum()

0

In [3]:
def cal_linear(iaqi_lo, iaqi_hi, bp_lo, bp_hi, cp):
    # 线性缩放
    iaqi = (iaqi_hi - iaqi_lo) * (cp - bp_lo) / (bp_hi - bp_lo) +iaqi_lo
    return iaqi


def cal_pm_iaqi(pm_val):
    # 计算PM2.5的IAQI
    if 0 <= pm_val < 36:
        iaqi = cal_linear(0, 50, 0, 35, pm_val)
    elif 36 <= pm_val < 76:
        iaqi = cal_linear(50, 100, 35, 75, pm_val)
    elif 76 <= pm_val < 116:
        iaqi = cal_linear(100, 150, 75, 115, pm_val)
    elif 116 <= pm_val < 151:
        iaqi = cal_linear(150, 200, 115, 150, pm_val)
    elif 151 <= pm_val < 251:
        iaqi = cal_linear(200, 300, 150, 250, pm_val)
    elif 251 <= pm_val < 351:
        iaqi = cal_linear(300, 400, 250, 350, pm_val)
    elif 351 <= pm_val < 501:
        iaqi = cal_linear(400, 500, 350, 500, pm_val)
    else:
        iaqi = None
    return iaqi


def cal_co_iaqi(co_val):
    # 计算co的IAQI
    if 0 <= co_val < 3:
        iaqi = cal_linear(0, 50, 0, 2, co_val)
    elif 3 <= co_val < 5:
        iaqi = cal_linear(50, 100, 2, 4, co_val)
    elif 5 <= co_val < 15:
        iaqi = cal_linear(100, 150, 4, 14, co_val)
    elif 15 <= co_val < 25:
        iaqi = cal_linear(150, 200, 14, 24, co_val)
    elif 25 <= co_val < 37:
        iaqi = cal_linear(200, 300, 24, 36, co_val)
    elif 37 <= co_val < 49:
        iaqi = cal_linear(300, 400, 36, 48, co_val)
    elif 49 <= co_val < 61:
        iaqi = cal_linear(400, 500, 48, 60, co_val)
    else:
        iaqi = None
    return iaqi


def cal_aqi(pm_val, co_val):
    pm_iaqi = cal_pm_iaqi(pm_val)
    co_iaqi = cal_co_iaqi(co_val)

    iaqi_list = []
    iaqi_list.append(pm_iaqi)
    iaqi_list.append(co_iaqi)

    for i in iaqi_list:
        if i == None:
            iaqi_list.remove(i)
    aqi = max(iaqi_list, default=None)
    return aqi

In [4]:
test['AQI'] = test.apply(lambda x: cal_aqi(x['PM2.5'], x['CO']), axis=1)
test

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station,datetime,AQI
0,1,2013,3,1,0,9.0,9.0,3.0,17.0,300.0,89.0,-0.5,1024.5,-21.4,0.0,NNW,5.7,Dongsi,2013-03-01 00:00:00,12.857143
1,2,2013,3,1,1,4.0,4.0,3.0,16.0,300.0,88.0,-0.7,1025.1,-22.1,0.0,NW,3.9,Dongsi,2013-03-01 01:00:00,5.714286
2,3,2013,3,1,2,7.0,7.0,NaN,17.0,300.0,60.0,-1.2,1025.3,-24.6,0.0,NNW,5.3,Dongsi,2013-03-01 02:00:00,10.000000
3,4,2013,3,1,3,3.0,3.0,5.0,18.0,NaN,NaN,-1.4,1026.2,-25.5,0.0,N,4.9,Dongsi,2013-03-01 03:00:00,4.285714
4,5,2013,3,1,4,3.0,3.0,7.0,NaN,200.0,84.0,-1.9,1027.1,-24.5,0.0,NNW,3.2,Dongsi,2013-03-01 04:00:00,4.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,35060,2017,2,28,19,16.0,51.0,3.0,29.0,400.0,73.0,12.5,1013.5,-16.2,0.0,NW,2.4,Dongsi,2017-02-28 19:00:00,22.857143
35060,35061,2017,2,28,20,18.0,45.0,3.0,43.0,500.0,54.0,11.6,1013.6,-15.1,0.0,WNW,0.9,Dongsi,2017-02-28 20:00:00,25.714286
35061,35062,2017,2,28,21,23.0,58.0,5.0,61.0,700.0,28.0,10.8,1014.2,-13.3,0.0,NW,1.1,Dongsi,2017-02-28 21:00:00,32.857143
35062,35063,2017,2,28,22,23.0,53.0,9.0,75.0,900.0,15.0,10.5,1014.4,-12.9,0.0,NNW,1.2,Dongsi,2017-02-28 22:00:00,32.857143


In [5]:
position = json.load(open('./air_quality/position.json', 'r', encoding='utf-8'))
position

{'success': True,
 'data': [{'city': '巴音郭楞州',
   'city_code': '0',
   'station': '孔雀公园',
   'station_code': '1956A',
   'lng': 86.1472,
   'lat': 41.7538},
  {'city': '巴音郭楞州',
   'city_code': '0',
   'station': '棉纺厂',
   'station_code': '1957A',
   'lng': 86.1972,
   'lat': 41.7166},
  {'city': '巴音郭楞州',
   'city_code': '0',
   'station': '经济开发区',
   'station_code': '1958A',
   'lng': 86.2357,
   'lat': 41.7173},
  {'city': '平度市',
   'city_code': '0',
   'station': '2号',
   'station_code': '1972A',
   'lng': 119.976,
   'lat': 36.799},
  {'city': '荣成市',
   'city_code': '0',
   'station': '山东成山轮胎公司',
   'station_code': '1980A',
   'lng': 122.41,
   'lat': 37.161},
  {'city': '文登市',
   'city_code': '0',
   'station': '文登市环保局',
   'station_code': '1981A',
   'lng': 122.038,
   'lat': 37.197},
  {'city': '乳山市',
   'city_code': '0',
   'station': '乳山市气象局',
   'station_code': '1984A',
   'lng': 121.544,
   'lat': 36.907},
  {'city': '昆山市',
   'city_code': '0',
   'station': '震川中学',
   'statio

In [6]:
position = pd.DataFrame.from_records(position['data'])
position

,city,city_code,station,station_code,lng,lat
0,巴音郭楞州,0,孔雀公园,1956A,86.1472,41.7538
1,巴音郭楞州,0,棉纺厂,1957A,86.1972,41.7166
2,巴音郭楞州,0,经济开发区,1958A,86.2357,41.7173
3,平度市,0,2号,1972A,119.9760,36.7990
4,荣成市,0,山东成山轮胎公司,1980A,122.4100,37.1610
...,...,...,...,...,...,...
1732,阿勒泰地区,654300,市站,2708A,88.1267,47.8515
1733,石河子市,659001,艾青诗歌馆,2709A,86.0497,44.2967
1734,石河子市,659001,阳光学校,2710A,86.0697,44.3075
1735,五家渠市,659004,农水大厦,2711A,87.5475,44.1756


In [7]:
BeiJing = position[position['city'] == '北京市']
print(BeiJing.dtypes)
BeiJing

city             object
city_code        object
station          object
station_code     object
lng             float64
lat             float64
dtype: object


,city,city_code,station,station_code,lng,lat
43,北京市,110000,万寿西宫,1001A,116.3660,39.8673
44,北京市,110000,定陵(对照点),1002A,116.1700,40.2865
45,北京市,110000,东四,1003A,116.4340,39.9522
46,北京市,110000,天坛,1004A,116.4340,39.8745
47,北京市,110000,农展馆,1005A,116.4730,39.9716
48,北京市,110000,官园,1006A,116.3610,39.9425
49,北京市,110000,海淀万柳,1007A,116.2878,39.9611
50,北京市,110000,顺义新城,1008A,116.7200,40.1438
51,北京市,110000,怀柔镇,1009A,116.6440,40.3937
52,北京市,110000,昌平镇,1010A,116.2300,40.1952


In [8]:
from math import radians, sin, cos, sqrt, atan2

def calculate_distance(row1, row2):
    # Convert latitude and longitude to radians
    lat1 = radians(row1['lat'])
    lon1 = radians(row1['lng'])
    lat2 = radians(row2['lat'])
    lon2 = radians(row2['lng'])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Earth's radius in meters
    radius = 6371000

    # Calculate x and y values
    x = radius * dlon * cos(lat1)
    y = radius * dlat

    return x, y




In [9]:
calculate_distance(BeiJing.iloc[0], BeiJing.iloc[1])

(-16727.730978826745, 46612.913249398756)